In [2]:
import numpy as np
import pandas as pd
from copy import copy

In [3]:
n = 16
m = 3
lambd = 1/6.403
mu = 1/0.412

In [4]:
sample = {'l' : 0, 't_sob' : 0, 'type' : 0, 'c' : 0, 't_min' : 0, 't_ozh' : 0, 'j' : 0, 'k' : 0}
device = {'j' : 0, 'k' : 0, 't_ost' : 0}

Задание 1

In [5]:
dT = 0.164
table = [{'l' : 0, 't_sob' : 0, 'type' : 0, 'c' : 0, 't_min' : 0, 't_ozh' : 0, 'j' : 0, 'k' : 0} for i in range(100)]
devices = [{'j' : 0, 'k' : i + 1, 't_ost' : 0} for i in range(n)]
table2 = []
table3 = [{'N' : i + 1, 't_zan' : 0, 't_pr' : 0, 'delta_pr' : 0} for i in range(n)]
table4 = [{'sost' : i, 'R_100' : 0, 'nu_100' : 0 , 'T_100' : 0,  'delta_100' : 0 } for i in range(101)]

for i in range(n):
    devices[i]['k'] = i + 1
queue = []
current_device = -1
time_cummulative = 0
min_time = 0
total_j = 0

In [6]:
for i in range(100):

    if current_device == -1:

        current_device = 0
        total_j += 1
        min_time = np.round(np.random.exponential(mu), 6)
        if i == 0:
            time_cummulative += dT
            table4[0]['T_100'] += dT
        else:
            time_cummulative += table[i-1]['t_ozh']
            table4[0]['T_100'] += table[i-1]['t_ozh']

        table4[0]['R_100'] += 1

        table[i]['l'] = i + 1
        table[i]['j'] = total_j
        table[i]['t_sob'] = time_cummulative
        table[i]['type'] = 1
        table[i]['c'] = 1
        table[i]['t_min'] = min_time
        table[i]['t_ozh'] = dT
        table[i]['k'] = 1

        devices[0]['j'] = total_j
        devices[0]['t_ost'] = min_time

        table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : 0, 't_och' : 0, 't_nob' : 0, 't_obs' : min_time,
                       't_kob' : 0, 'k_j' : 1})


    elif min_time < table[i-1]['t_ozh']:

        time_cummulative += min_time

        for j in range(n):
            if devices[j]['j'] != 0:
                devices[j]['t_ost'] -= min_time
                table3[j]['t_zan'] += min_time
        table4[table[i-1]['c']]['T_100'] += min_time
        table4[table[i-1]['c']]['R_100'] += 1

        table[i]['l'] = i + 1
        table[i]['j'] = devices[current_device]['j']
        table[i]['t_sob'] = time_cummulative
        table[i]['t_ozh'] = np.round(table[i-1]['t_ozh'] - min_time, 6)
        table[i]['type'] = 2
        table[i]['c'] = table[i-1]['c'] - 1
        table[i]['k'] = current_device + 1

        table2[table[i]['j'] - 1]['t_kob'] = np.round(time_cummulative, 6)
        table2[table[i]['j'] - 1]['t_nob'] = np.round(time_cummulative - table2[table[i]['j']-1]['t_obs'], 6)
        table2[table[i]['j']- 1]['t_och'] = np.round(table2[ table[i]['j'] - 1]['t_nob']  - table2[table[i]['j']-1]['t_z'], 6)


        devices[current_device]['j'] = 0
        devices[current_device]['t_ost'] = 0

        if len(queue) > 0:
            tmp = queue.pop(0)
            devices[current_device]['j'] = tmp[0]
            devices[current_device]['t_ost'] = tmp[1]
            table2[tmp[0] - 1]['k_j'] = current_device + 1

        current_device = -1
        min_time = np.inf
        for j in range(n):
            if devices[j]['j'] != 0 and min_time > devices[j]['t_ost']:
                min_time = devices[j]['t_ost']
                current_device = j
        if current_device == -1:
            table[i]['t_min'] = -1
        else:
            table[i]['t_min'] = min_time

    else:
        min_time -= table[i-1]['t_ozh']
        time_cummulative += table[i - 1]['t_ozh']
        total_j += 1
        for j in range(n):
            if devices[j]['j'] != 0:
                devices[j]['t_ost'] -= table[i - 1]['t_ozh']
                table3[j]['t_zan'] += table[i - 1]['t_ozh']

        table4[table[i-1]['c']]['T_100'] += table[i-1]['t_ozh']
        table4[table[i-1]['c']]['R_100'] += 1


        table[i]['l'] = i + 1
        table[i]['j'] = total_j
        table[i]['t_sob'] = time_cummulative
        table[i]['t_ozh'] = dT
        table[i]['type'] = 1
        table[i]['c'] = table[i-1]['c'] + 1

        counter = -1
        for j in range(n):
            if devices[j]['j'] == 0:
                counter = j
                break


        if counter == -1:
            queue.append([total_j, np.round(np.random.exponential(mu), 6)])
            table[i]['k'] = -1
            table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : len(queue), 't_och' : 0, 't_nob' : 0, 't_obs' : min_time,
                       't_kob' : 0, 'k_j' : -1})
        else:
            devices[counter]['j'] = total_j
            devices[counter]['t_ost'] = np.round(np.random.exponential(mu), 6)
            if devices[counter]['t_ost'] < min_time:
                min_time = devices[counter]['t_ost']
                current_device = counter
            table[i]['k'] = counter + 1
            table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : 0, 't_och' : 0, 't_nob' : 0, 't_obs' : devices[counter]['t_ost'],
                       't_kob' : 0, 'k_j' : counter + 1})

        table[i]['t_min'] = min_time


end_service_flg = False
i = 100

while True:

    if len(queue) == 0:
        end_service_flg = True
        for j in range(n):
            if devices[j]['j'] != 0:
                end_service_flg = False
                break
        if end_service_flg:
            break


    time_cummulative += min_time
    table.append({'l' : 0, 't_sob' : 0, 'type' : 0, 'c' : 0, 't_min' : 0, 't_ozh' : 0, 'j' : 0, 'k' : 0})

    for j in range(n):
        if devices[j]['j'] != 0:
            devices[j]['t_ost'] -= min_time

    table[i]['l'] = i + 1
    table[i]['j'] = devices[current_device]['j']
    table[i]['t_sob'] = time_cummulative
    table[i]['t_ozh'] = np.round(table[i-1]['t_ozh'] - min_time, 6)
    table[i]['type'] = 2
    table[i]['c'] = table[i-1]['c'] - 1
    table[i]['k'] = current_device + 1

    table2[table[i]['j'] - 1]['t_kob'] = np.round(time_cummulative, 6)
    table2[table[i]['j'] - 1]['t_nob'] = np.round(time_cummulative - table2[table[i]['j']-1]['t_obs'], 6)
    table2[table[i]['j']- 1]['t_och'] = np.round(table2[ table[i]['j'] - 1]['t_nob']  - table2[table[i]['j']-1]['t_z'], 6)


    devices[current_device]['j'] = 0
    devices[current_device]['t_ost'] = 0

    if len(queue) > 0:
        tmp = queue.pop(0)
        devices[current_device]['j'] = tmp[0]
        devices[current_device]['t_ost'] = tmp[1]
        table2[tmp[0] - 1]['k_j'] = current_device + 1

    current_device = -1
    min_time = np.inf
    for j in range(n):
        if devices[j]['j'] != 0 and min_time > devices[j]['t_ost']:
            min_time = devices[j]['t_ost']
            current_device = j
    if current_device == -1:
        table[i]['t_min'] = -1
    else:
        table[i]['t_min'] = min_time

    i += 1


t100 = table[99]['t_sob']
for i in range(n):
    table3[i]['t_pr'] = t100 - table3[i]['t_zan']
    table3[i]['delta_pr'] = table3[i]['t_pr']/t100

notnullindex = 0
for i in range(101):
    if table4[i]['R_100'] != 0:
        notnullindex = i

table4 = table4[:notnullindex + 1]

for i in range(len(table4)):
    table4[i]['nu_100'] = table4[i]['R_100']/100
    table4[i]['delta_100'] = table4[i]['T_100']/t100

table3.append({'N' : '', 't_zan' : 1/n * np.sum([table3[i]['t_zan'] for i in range(n)]),
              't_pr' : 1/n * np.sum([table3[i]['t_pr'] for i in range(n)]),
              'delta_pr' : 1/n * np.sum([table3[i]['delta_pr'] for i in range(n)])})

table4.append({'sost' : '', 'R_100' : np.sum([table4[i]['R_100'] for i in range(len(table4))]),
              'nu_100' : np.sum([table4[i]['nu_100'] for i in range(len(table4))]),
              'T_100' : np.sum([table4[i]['T_100'] for i in range(len(table4))]),
              'delta_100' : np.sum([table4[i]['delta_100'] for i in range(len(table4))])})

In [7]:
table = table[:100]
table_df = pd.DataFrame.from_dict(table)

In [8]:
table_df.head()

,l,t_sob,type,c,t_min,t_ozh,j,k
0,1,0.164,1,1,7.899937,0.164,1,1
1,2,0.328,1,2,5.553318,0.164,2,2
2,3,0.492,1,3,1.917907,0.164,3,3
3,4,0.656,1,4,1.753907,0.164,4,4
4,5,0.820,1,5,1.589907,0.164,5,5


In [9]:
table_df.to_csv('table1_1.csv', index = False)

In [10]:
table_df.tail()

,l,t_sob,type,c,t_min,t_ozh,j,k
95,96,9.184000,1,16,0.003518,0.164000,56,13
96,97,9.187518,2,15,0.297893,0.160482,54,16
97,98,9.348000,1,16,0.137411,0.164000,57,16
98,99,9.485411,2,15,0.089002,0.026589,21,12
99,100,9.512000,1,16,0.062413,0.164000,58,12


In [11]:
table_2_df = pd.DataFrame.from_dict(table2)

In [12]:
table_2_df.to_csv('table1_2.csv', index = False)

In [13]:
table_3_df = pd.DataFrame.from_dict(table3)
table_3_df.to_csv('table1_3.csv', index = False)
table_4_df = pd.DataFrame.from_dict(table4)
table_4_df.to_csv('table1_4.csv', index = False)

In [14]:
table_3_df

,N,t_zan,t_pr,delta_pr
0,1,9.211937,0.300063,0.031546
1,2,9.161318,0.350682,0.036867
2,3,8.870414,0.641586,0.067450
3,4,8.762143,0.749857,0.078833
4,5,8.163195,1.348805,0.141800
5,6,8.333695,1.178305,0.123876
6,7,7.488752,2.023248,0.212705
7,8,6.501940,3.010060,0.316449
8,9,6.264375,3.247625,0.341424
9,10,6.810442,2.701558,0.284016


In [15]:
table_4_df

,sost,R_100,nu_100,T_100,delta_100
0,0,1,0.01,0.164000,0.017241
1,1,1,0.01,0.164000,0.017241
2,2,1,0.01,0.164000,0.017241
3,3,1,0.01,0.164000,0.017241
4,4,1,0.01,0.164000,0.017241
5,5,1,0.01,0.164000,0.017241
6,6,1,0.01,0.164000,0.017241
7,7,1,0.01,0.164000,0.017241
8,8,6,0.06,0.455188,0.047854
9,9,11,0.11,0.934115,0.098204


In [16]:
serv_demands = 57
print(f"Count of served demands: {serv_demands}")
z = 0
for i in range(100):
    cnt = 0
    t_event = table[i]['t_sob']
    for j in range(len(table1)):
        if table1[j]['t_z'] >= t_event:
            cnt += 1
    z += cnt
print(f"z(100): {z/100}")

sum_q_time = 0
sum_stay_time = 0
for i in range(serv_demands):
    sum_q_time += table1[i]['t_och']
    sum_stay_time += (table1[i]['t_kob'] - table1[i]['t_z'])
q_time_mean = sum_q_time / serv_demands
print(f"q_time_mean: {round(q_time_mean, 5)}")
stay_time_mean = sum_stay_time / serv_demands
print(f"stay_time_mean: {round(stay_time_mean, 5)}")

Count of served demands: 57


<class 'NameError'>: name 'table1' is not defined

Задание 2

In [17]:
table = [{'l' : 0, 't_sob' : 0, 'type' : 0, 'c' : 0, 't_min' : 0, 't_ozh' : 0, 'j' : 0, 'k' : 0} for i in range(100)]
devices = [{'j' : 0, 'k' : i + 1, 't_ost' : 0} for i in range(n)]
table2 = []
table3 = [{'N' : i + 1, 't_zan' : 0, 't_pr' : 0, 'delta_pr' : 0} for i in range(n)]
table4 = [{'sost' : i, 'R_100' : 0, 'nu_100' : 0 , 'T_100' : 0,  'delta_100' : 0 } for i in range(101)]

for i in range(n):
    devices[i]['k'] = i + 1
queue = []
current_device = -1
time_cummulative = 0
min_time = 0
total_j = 0

In [18]:
for i in range(100):

    if current_device == -1:

        dT =  np.round(np.random.exponential(lambd), 6)
        current_device = 0
        total_j += 1
        min_time = np.round(np.random.exponential(mu), 6)
        if i == 0:
            time_cummulative += dT
            table4[0]['T_100'] += dT
        else:
            time_cummulative += table[i-1]['t_ozh']
            table4[0]['T_100'] += table[i-1]['t_ozh']

        table4[0]['R_100'] += 1

        table[i]['l'] = i + 1
        table[i]['j'] = total_j
        table[i]['t_sob'] = time_cummulative
        table[i]['type'] = 1
        table[i]['c'] = 1
        table[i]['t_min'] = min_time
        table[i]['t_ozh'] = dT
        table[i]['k'] = 1

        devices[0]['j'] = total_j
        devices[0]['t_ost'] = min_time

        table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : 0, 't_och' : 0, 't_nob' : 0, 't_obs' : min_time,
                       't_kob' : 0, 'k_j' : 1})


    elif min_time < table[i-1]['t_ozh']:

        time_cummulative += min_time

        for j in range(n):
            if devices[j]['j'] != 0:
                devices[j]['t_ost'] -= min_time
                table3[j]['t_zan'] += min_time
        table4[table[i-1]['c']]['T_100'] += min_time
        table4[table[i-1]['c']]['R_100'] += 1

        table[i]['l'] = i + 1
        table[i]['j'] = devices[current_device]['j']
        table[i]['t_sob'] = time_cummulative
        table[i]['t_ozh'] = np.round(table[i-1]['t_ozh'] - min_time, 6)
        table[i]['type'] = 2
        table[i]['c'] = table[i-1]['c'] - 1
        table[i]['k'] = current_device + 1

        table2[table[i]['j'] - 1]['t_kob'] = np.round(time_cummulative, 6)
        table2[table[i]['j'] - 1]['t_nob'] = np.round(time_cummulative - table2[table[i]['j']-1]['t_obs'], 6)
        table2[table[i]['j']- 1]['t_och'] = np.round(table2[ table[i]['j'] - 1]['t_nob']  - table2[table[i]['j']-1]['t_z'], 6)


        devices[current_device]['j'] = 0
        devices[current_device]['t_ost'] = 0

        if len(queue) > 0:
            tmp = queue.pop(0)
            devices[current_device]['j'] = tmp[0]
            devices[current_device]['t_ost'] = tmp[1]
            table2[tmp[0] - 1]['k_j'] = current_device + 1

        current_device = -1
        min_time = np.inf
        for j in range(n):
            if devices[j]['j'] != 0 and min_time > devices[j]['t_ost']:
                min_time = devices[j]['t_ost']
                current_device = j
        if current_device == -1:
            table[i]['t_min'] = -1
        else:
            table[i]['t_min'] = min_time

    else:
        dT =  np.round(np.random.exponential(lambd), 6)
        min_time -= table[i-1]['t_ozh']
        time_cummulative += table[i - 1]['t_ozh']
        total_j += 1
        for j in range(n):
            if devices[j]['j'] != 0:
                devices[j]['t_ost'] -= table[i - 1]['t_ozh']
                table3[j]['t_zan'] += table[i - 1]['t_ozh']

        table4[table[i-1]['c']]['T_100'] += table[i-1]['t_ozh']
        table4[table[i-1]['c']]['R_100'] += 1


        table[i]['l'] = i + 1
        table[i]['j'] = total_j
        table[i]['t_sob'] = time_cummulative
        table[i]['t_ozh'] = dT
        table[i]['type'] = 1
        table[i]['c'] = table[i-1]['c'] + 1

        counter = -1
        for j in range(n):
            if devices[j]['j'] == 0:
                counter = j
                break


        if counter == -1:
            queue.append([total_j, np.round(np.random.exponential(mu), 6)])
            table[i]['k'] = -1
            table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : len(queue), 't_och' : 0, 't_nob' : 0, 't_obs' : min_time,
                       't_kob' : 0, 'k_j' : -1})
        else:
            devices[counter]['j'] = total_j
            devices[counter]['t_ost'] = np.round(np.random.exponential(mu), 6)
            if devices[counter]['t_ost'] < min_time:
                min_time = devices[counter]['t_ost']
                current_device = counter
            table[i]['k'] = counter + 1
            table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : 0, 't_och' : 0, 't_nob' : 0, 't_obs' : devices[counter]['t_ost'],
                       't_kob' : 0, 'k_j' : counter + 1})

        table[i]['t_min'] = min_time


end_service_flg = False
i = 100

while True:

    if len(queue) == 0:
        end_service_flg = True
        for j in range(n):
            if devices[j]['j'] != 0:
                end_service_flg = False
                break
        if end_service_flg:
            break


    time_cummulative += min_time
    table.append({'l' : 0, 't_sob' : 0, 'type' : 0, 'c' : 0, 't_min' : 0, 't_ozh' : 0, 'j' : 0, 'k' : 0})

    for j in range(n):
        if devices[j]['j'] != 0:
            devices[j]['t_ost'] -= min_time

    table[i]['l'] = i + 1
    table[i]['j'] = devices[current_device]['j']
    table[i]['t_sob'] = time_cummulative
    table[i]['t_ozh'] = np.round(table[i-1]['t_ozh'] - min_time, 6)
    table[i]['type'] = 2
    table[i]['c'] = table[i-1]['c'] - 1
    table[i]['k'] = current_device + 1

    table2[table[i]['j'] - 1]['t_kob'] = np.round(time_cummulative, 6)
    table2[table[i]['j'] - 1]['t_nob'] = np.round(time_cummulative - table2[table[i]['j']-1]['t_obs'], 6)
    table2[table[i]['j']- 1]['t_och'] = np.round(table2[ table[i]['j'] - 1]['t_nob']  - table2[table[i]['j']-1]['t_z'], 6)


    devices[current_device]['j'] = 0
    devices[current_device]['t_ost'] = 0

    if len(queue) > 0:
        tmp = queue.pop(0)
        devices[current_device]['j'] = tmp[0]
        devices[current_device]['t_ost'] = tmp[1]
        table2[tmp[0] - 1]['k_j'] = current_device + 1

    current_device = -1
    min_time = np.inf
    for j in range(n):
        if devices[j]['j'] != 0 and min_time > devices[j]['t_ost']:
            min_time = devices[j]['t_ost']
            current_device = j
    if current_device == -1:
        table[i]['t_min'] = -1
    else:
        table[i]['t_min'] = min_time

    i += 1


t100 = table[99]['t_sob']
for i in range(n):
    table3[i]['t_pr'] = t100 - table3[i]['t_zan']
    table3[i]['delta_pr'] = table3[i]['t_pr']/t100

notnullindex = 0
for i in range(101):
    if table4[i]['R_100'] != 0:
        notnullindex = i

table4 = table4[:notnullindex + 1]

for i in range(len(table4)):
    table4[i]['nu_100'] = table4[i]['R_100']/100
    table4[i]['delta_100'] = table4[i]['T_100']/t100

table3.append({'N' : '', 't_zan' : 1/n * np.sum([table3[i]['t_zan'] for i in range(n)]),
              't_pr' : 1/n * np.sum([table3[i]['t_pr'] for i in range(n)]),
              'delta_pr' : 1/n * np.sum([table3[i]['delta_pr'] for i in range(n)])})

table4.append({'sost' : '', 'R_100' : np.sum([table4[i]['R_100'] for i in range(len(table4))]),
              'nu_100' : np.sum([table4[i]['nu_100'] for i in range(len(table4))]),
              'T_100' : np.sum([table4[i]['T_100'] for i in range(len(table4))]),
              'delta_100' : np.sum([table4[i]['delta_100'] for i in range(len(table4))])})

In [74]:
table = table[:100]
table_df = pd.DataFrame.from_dict(table)
table2_df = pd.DataFrame.from_dict(table2)
table3_df = pd.DataFrame.from_dict(table3)
table4_df = pd.DataFrame.from_dict(table4)
table_df.to_csv('table2_1.csv')
table2_df.to_csv('table2_2.csv')
table3_df.to_csv('table2_3.csv')
table4_df.to_csv('table2_4.csv')
print (table2_df)

     j       t_z  q_j     t_och     t_nob      t_obs      t_kob  k_j
0    1  0.171698    0  0.000000  0.171698   2.035662   2.207360    1
1    2  0.343396    0  0.000000  0.343396   0.883227   1.226623    2
2    3  0.448758    0  0.000000  0.448758   3.891523   4.340281    3
3    4  0.855087    0  0.000000  0.855087   1.893065   2.748152    4
4    5  0.953087    0  0.000000  0.953087   4.531320   5.484407    5
5    6  1.164631    0  0.000000  1.164631   0.333265   1.497896    6
6    7  1.240799    0  0.000000  1.240799   1.144557   2.385356    2
7    8  1.260523    0  0.000000  1.260523   4.489412   5.749935    7
8    9  1.598124    0  0.000000  1.598124   2.734887   4.333011    6
9   10  1.641647    0  0.000000  1.641647   1.807867   3.449514    8
10  11  1.665834    0  0.000000  1.665834   0.345606   2.011440    9
11  12  1.733142    0  0.000000  1.733142   1.460102   3.193244   10
12  13  1.853877    0  0.000000  1.853877   3.914556   5.768433   11
13  14  2.064378    0  0.000000  2

Задача 3

In [20]:
table = [{'l' : 0, 't_sob' : 0, 'type' : 0, 'c' : 0, 't_min' : 0, 't_ozh' : 0, 'j' : 0, 'k' : 0} for i in range(100)]
devices = [{'j' : 0, 'k' : i + 1, 't_ost' : 0} for i in range(n)]
table2 = []
table3 = [{'N' : i + 1, 't_zan' : 0, 't_pr' : 0, 'delta_pr' : 0} for i in range(n)]
table4 = [{'sost' : i, 'R_100' : 0, 'nu_100' : 0 , 'T_100' : 0,  'delta_100' : 0 } for i in range(101)]

for i in range(n):
    devices[i]['k'] = i + 1
queue = []
current_device = -1
time_cummulative = 0
min_time = 0
total_j = 0

In [21]:
for i in range(100):

    if current_device == -1:

        dT =  np.round(np.random.exponential(lambd), 6)
        current_device = 0
        total_j += 1
        min_time = np.round(np.random.exponential(mu), 6)
        if i == 0:
            time_cummulative += dT
            table4[0]['T_100'] += dT
        else:
            time_cummulative += table[i-1]['t_ozh']
            table4[0]['T_100'] += table[i-1]['t_ozh']

        table4[0]['R_100'] += 1

        table[i]['l'] = i + 1
        table[i]['j'] = total_j
        table[i]['t_sob'] = time_cummulative
        table[i]['type'] = 1
        table[i]['c'] = 1
        table[i]['t_min'] = min_time
        table[i]['t_ozh'] = dT
        table[i]['k'] = 1

        devices[0]['j'] = total_j
        devices[0]['t_ost'] = min_time

        table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : 0, 't_och' : 0, 't_nob' : 0, 't_obs' : min_time,
                       't_kob' : 0, 'k_j' : 1})


    elif min_time < table[i-1]['t_ozh']:

        time_cummulative += min_time

        for j in range(n):
            if devices[j]['j'] != 0:
                devices[j]['t_ost'] -= min_time
                table3[j]['t_zan'] += min_time
        table4[table[i-1]['c']]['T_100'] += min_time
        table4[table[i-1]['c']]['R_100'] += 1

        table[i]['l'] = i + 1
        table[i]['j'] = devices[current_device]['j']
        table[i]['t_sob'] = time_cummulative
        table[i]['t_ozh'] = np.round(table[i-1]['t_ozh'] - min_time, 6)
        table[i]['type'] = 3
        table[i]['c'] = table[i-1]['c'] - 1
        table[i]['k'] = current_device + 1

        table2[table[i]['j'] - 1]['t_kob'] = np.round(time_cummulative, 6)
        table2[table[i]['j'] - 1]['t_nob'] = np.round(time_cummulative - table2[table[i]['j']-1]['t_obs'], 6)
        table2[table[i]['j']- 1]['t_och'] = np.round(table2[ table[i]['j'] - 1]['t_nob']  - table2[table[i]['j']-1]['t_z'], 6)


        devices[current_device]['j'] = 0
        devices[current_device]['t_ost'] = 0

        if len(queue) > 0:
            tmp = queue.pop(0)
            devices[current_device]['j'] = tmp[0]
            devices[current_device]['t_ost'] = tmp[1]
            table2[tmp[0] - 1]['k_j'] = current_device + 1

        current_device = -1
        min_time = np.inf
        for j in range(n):
            if devices[j]['j'] != 0 and min_time > devices[j]['t_ost']:
                min_time = devices[j]['t_ost']
                current_device = j
        if current_device == -1:
            table[i]['t_min'] = -1
        else:
            table[i]['t_min'] = min_time

    else:
        dT =  np.round(np.random.exponential(lambd), 6)
        min_time -= table[i-1]['t_ozh']
        time_cummulative += table[i - 1]['t_ozh']
        total_j += 1
        for j in range(n):
            if devices[j]['j'] != 0:
                devices[j]['t_ost'] -= table[i - 1]['t_ozh']
                table3[j]['t_zan'] += table[i - 1]['t_ozh']

        table4[table[i-1]['c']]['T_100'] += table[i-1]['t_ozh']
        table4[table[i-1]['c']]['R_100'] += 1


        table[i]['l'] = i + 1
        table[i]['j'] = total_j
        table[i]['t_sob'] = time_cummulative
        table[i]['t_ozh'] = dT
        table[i]['type'] = 1
        table[i]['c'] = table[i-1]['c'] + 1

        counter = -1
        for j in range(n):
            if devices[j]['j'] == 0:
                counter = j
                break


        if counter == -1:
            if len(queue) < m:
                queue.append([total_j, np.round(np.random.exponential(mu), 6)])
                table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : len(queue), 't_och' : 0, 't_nob' : 0, 't_obs' : min_time,
                       't_kob' : 0, 'k_j' : -1})
            else:
                table[i]['type'] = 2
                table[i]['c'] = table[i-1]['c']
                table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : -1, 't_och' : 0, 't_nob' : -1, 't_obs' : 0,
                       't_kob' : time_cummulative, 'k_j' : -2})
            table[i]['k'] = -1
        else:
            devices[counter]['j'] = total_j
            devices[counter]['t_ost'] = np.round(np.random.exponential(mu), 6)
            if devices[counter]['t_ost'] < min_time:
                min_time = devices[counter]['t_ost']
                current_device = counter
            table[i]['k'] = counter + 1
            table2.append({'j' : total_j, 't_z' : time_cummulative, 'q_j' : 0, 't_och' : 0, 't_nob' : 0, 't_obs' : devices[counter]['t_ost'],
                       't_kob' : 0, 'k_j' : counter + 1})

        table[i]['t_min'] = min_time


end_service_flg = False
i = 100

while True:

    if len(queue) == 0:
        end_service_flg = True
        for j in range(n):
            if devices[j]['j'] != 0:
                end_service_flg = False
                break
        if end_service_flg:
            break


    time_cummulative += min_time
    table.append({'l' : 0, 't_sob' : 0, 'type' : 0, 'c' : 0, 't_min' : 0, 't_ozh' : 0, 'j' : 0, 'k' : 0})

    for j in range(n):
        if devices[j]['j'] != 0:
            devices[j]['t_ost'] -= min_time

    table[i]['l'] = i + 1
    table[i]['j'] = devices[current_device]['j']
    table[i]['t_sob'] = time_cummulative
    table[i]['t_ozh'] = np.round(table[i-1]['t_ozh'] - min_time, 6)
    table[i]['type'] = 2
    table[i]['c'] = table[i-1]['c'] - 1
    table[i]['k'] = current_device + 1

    table2[table[i]['j'] - 1]['t_kob'] = np.round(time_cummulative, 6)
    table2[table[i]['j'] - 1]['t_nob'] = np.round(time_cummulative - table2[table[i]['j']-1]['t_obs'], 6)
    table2[table[i]['j']- 1]['t_och'] = np.round(table2[ table[i]['j'] - 1]['t_nob']  - table2[table[i]['j']-1]['t_z'], 6)


    devices[current_device]['j'] = 0
    devices[current_device]['t_ost'] = 0

    if len(queue) > 0:
        tmp = queue.pop(0)
        devices[current_device]['j'] = tmp[0]
        devices[current_device]['t_ost'] = tmp[1]
        table2[tmp[0] - 1]['k_j'] = current_device + 1

    current_device = -1
    min_time = np.inf
    for j in range(n):
        if devices[j]['j'] != 0 and min_time > devices[j]['t_ost']:
            min_time = devices[j]['t_ost']
            current_device = j
    if current_device == -1:
        table[i]['t_min'] = -1
    else:
        table[i]['t_min'] = min_time

    i += 1


t100 = table[99]['t_sob']
for i in range(n):
    table3[i]['t_pr'] = t100 - table3[i]['t_zan']
    table3[i]['delta_pr'] = table3[i]['t_pr']/t100

notnullindex = 0
for i in range(101):
    if table4[i]['R_100'] != 0:
        notnullindex = i

table4 = table4[:notnullindex + 1]

for i in range(len(table4)):
    table4[i]['nu_100'] = table4[i]['R_100']/100
    table4[i]['delta_100'] = table4[i]['T_100']/t100

table3.append({'N' : '', 't_zan' : 1/n * np.sum([table3[i]['t_zan'] for i in range(n)]),
              't_pr' : 1/n * np.sum([table3[i]['t_pr'] for i in range(n)]),
              'delta_pr' : 1/n * np.sum([table3[i]['delta_pr'] for i in range(n)])})

table4.append({'sost' : '', 'R_100' : np.sum([table4[i]['R_100'] for i in range(len(table4))]),
              'nu_100' : np.sum([table4[i]['nu_100'] for i in range(len(table4))]),
              'T_100' : np.sum([table4[i]['T_100'] for i in range(len(table4))]),
              'delta_100' : np.sum([table4[i]['delta_100'] for i in range(len(table4))])})

In [22]:
table = table[:100]
table_df = pd.DataFrame.from_dict(table)
table2_df = pd.DataFrame.from_dict(table2)
table3_df = pd.DataFrame.from_dict(table3)
table4_df = pd.DataFrame.from_dict(table4)
table_df.to_csv('table3_1.csv', index = False)
table2_df.to_csv('table3_2.csv', index = False)
table3_df.to_csv('table3_3.csv', index = False)
table4_df.to_csv('table3_4.csv', index = False)

In [86]:
import math
n=16
m=3
lam=6.403
mu=0.412
ro=lam/mu
nu=ro/n
#xy=1+(ro/n)+(ro/n)**2+(ro/n)**3+(ro/n)**4+(ro/n)**5+(ro/n)**6+(ro/n)**7+(ro/n)**8+(ro/n)**9+(ro/n)**10
##r0=(1+ro+(ro**2)/(math.factorial(2))+(ro**3)/(math.factorial(3))+(ro**4)/(math.factorial(4))+(ro**5)/(math.factorial(5))+(ro**6)/(math.factorial(6))+(ro**7)/(math.factorial(7))+(ro**8)/(math.factorial(8))+(ro**9)*xy/(math.factorial(9))+(ro**10)/(math.factorial(10))+(ro**11)/(math.factorial(11))*xy)**(-1)
r0=(1+ro+(ro**2)/(math.factorial(2))+(ro**3)/(math.factorial(3))+(ro**4)/(math.factorial(4))+(ro**5)/(math.factorial(5))+(ro**6)/(math.factorial(6))+(ro**7)/(math.factorial(7))+(ro**8)/(math.factorial(8))+(ro**9)/(math.factorial(9))+ (ro**10)/(math.factorial(10))+(ro**11)/(math.factorial(11))+(ro**12)/(math.factorial(12))+(ro**13)/(math.factorial(13))+(ro**14)/(math.factorial(14))+(ro**15)/(math.factorial(15))+(ro**16)/(math.factorial(16))+(ro**17)/(math.factorial(17))+(ro**18)/(math.factorial(18))+(ro**19)/(math.factorial(19))+(ro**20)/(math.factorial(20))+(ro**21)/(math.factorial(21))+(ro**22)/(math.factorial(22))+(ro**23)/(math.factorial(23))+(ro**24)/(math.factorial(24))+(ro**25)/(math.factorial(25))+(ro**26)/(math.factorial(26))+(ro**27)/(math.factorial(27))+(ro**28)/(math.factorial(28))+(ro**29)/(math.factorial(29))*(1/(1-nu)))**(-1)
##r9=(ro**9*r0)/(math.factorial(9))
sum=0
print("r0=", r0)
for k in range(0,30):
    rk=r0*ro**k/math.factorial(k)
    sum+=rk
    print("r",k,"=",rk)
sum-=r0
print(sum)

r0= 1.7392577712170334e-07
r 0 = 1.7392577712170334e-07
r 1 = 2.703026094442394e-06
r 2 = 2.1004218546983798e-05
r 3 = 0.00010881068879962561
r 4 = 0.0004227638594563123
r 5 = 0.0013140567922809551
r 6 = 0.0034036835117212598
r 7 = 0.007556791097625253
r 8 = 0.014680258919324785
r 9 = 0.02534997245427092
r 10 = 0.03939705670502347
r 11 = 0.05566181687605147
r 12 = 0.07208790725270177
r 13 = 0.08617977411109959
r 14 = 0.09566731859108368
r 15 = 0.09911939173765515
r 16 = 0.09627752810925452
r 17 = 0.08801613542026795
r 18 = 0.07599343515317902
r 19 = 0.06215967875393527
r 20 = 0.04830199308998149
r 21 = 0.03574637791899578
r 22 = 0.025251992256766317
r 23 = 0.017062949178986358
r 24 = 0.01104915691677282
r 25 = 0.006868713760980229
r 26 = 0.004105710811385027
r 27 = 0.0023632565916305577
r 28 = 0.0013117139351777444
r 29 = 0.0007029548315151572
0.9761849065705638


In [94]:
import math
n=16
m=3
lam=6.403
mu=0.412
ro=lam/mu
#xy=1+(ro/n)+(ro/n)**2+(ro/n)**3+(ro/n)**4+(ro/n)**5+(ro/n)**6+(ro/n)**7+(ro/n)**8+(ro/n)**9+(ro/n)**10
##r0=(1+ro+(ro**2)/(math.factorial(2))+(ro**3)/(math.factorial(3))+(ro**4)/(math.factorial(4))+(ro**5)/(math.factorial(5))+(ro**6)/(math.factorial(6))+(ro**7)/(math.factorial(7))+(ro**8)/(math.factorial(8))+(ro**9)*xy/(math.factorial(9))+(ro**10)/(math.factorial(10))+(ro**11)/(math.factorial(11))*xy)**(-1)
r0=(1+ro+(ro**2)/(math.factorial(2))+(ro**3)/(math.factorial(3))+(ro**4)/(math.factorial(4))+(ro**5)/(math.factorial(5))+(ro**6)/(math.factorial(6))+(ro**7)/(math.factorial(7))+(ro**8)/(math.factorial(8))+(ro**9)/(math.factorial(9))+ (ro**10)/(math.factorial(10))+(ro**11)/(math.factorial(11))+(ro**12)/(math.factorial(12))+(ro**13)/(math.factorial(13))+(ro**14)/(math.factorial(14))+(ro**15)/(math.factorial(15))+(ro**16)/(math.factorial(16))*(1+nu +nu**2 + nu**3))**(-1)
##r9=(ro**9*r0)/(math.factorial(9))
sum=0
print("r0=", r0)
for k in range(0,17):
    rk=r0*ro**k/math.factorial(k)
    sum+=rk
    print("r",k,"=",rk)
r_17=nu*r0*ro**16/math.factorial(16)
r_18=nu**2 * r0*ro**16/math.factorial(16)
r_19=nu**3 * r0*ro**16/math.factorial(16)
print("r",17,"=",r_17)
print("r",18,"=",r_18)
print("r",19,"=",r_19)
sum+=r_17+r_18+r_19
print(sum)

r0= 1.9995247163715275e-07
r 0 = 1.9995247163715275e-07
r 1 = 3.1075137764385656e-06
r 2 = 2.414734309531084e-05
r 3 = 0.00012509339631009328
r 4 = 0.00048602731588199476
r 5 = 0.0015106955842681612
r 6 = 0.0039130193471153055
r 7 = 0.008687608488064946
r 8 = 0.016877050105910146
r 9 = 0.02914340664189392
r 10 = 0.04529253221554532
r 11 = 0.06399119236013606
r 12 = 0.0828753245715921
r 13 = 0.09907593413590442
r 14 = 0.10998321884053328
r 15 = 0.11395186897021596
r 16 = 0.11068474165902499
r 17 = 0.10751128653560936
r 18 = 0.10442881791376012
r 19 = 0.10143472710889047
1.0
